In [1]:
import pandas as pd
import numpy as np
from geopy import distance

In [2]:
dvf_geocoordinates = pd.read_csv('dvf-geocoordinates.csv')

C:\Users\JouJo\Anaconda3\envs\Python3_9\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (17) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
dvf_geocoordinates = dvf_geocoordinates.dropna(subset=['adresse'])

In [4]:
ratp = pd.read_csv('ratp-geocoordinates.csv').query("stop_lat.notna()")

In [5]:
ratp_coords = []
ratp_index = []
for i in ratp.index:
    ratp_index.append(i)
    lat = ratp.stop_lat[i]
    lon = ratp.stop_lon[i]
    ratp_coords.append((lat, lon))

In [6]:
dvf_coords = []
for i in dvf_geocoordinates.index:
    lat = dvf_geocoordinates.latitude[i]
    lon = dvf_geocoordinates.longitude[i]
    dvf_coords.append((lat, lon))

In [7]:
def calculate_distance(center, points):
    return list(map(lambda x: distance.distance(center, x).km, points))

def count_points_in_radius(radius, distances):
    return len(list(filter(lambda x: x < radius, distances)))

def get_min_distance(distances):
    return min(distances)

def get_individual_info(center, points):
    distances = calculate_distance(center, points)
    ngare1 = count_points_in_radius(1, distances)
    ngare2 = count_points_in_radius(2, distances)
    ngare3 = count_points_in_radius(3, distances)
    min_distance = get_min_distance(distances)
    return ngare1, ngare2, ngare3, min_distance

In [8]:
from tqdm import tqdm

In [9]:
%%time
ratp_info = [None] * len(dvf_coords)

for i in tqdm(range(len(dvf_coords))):
    try:
        ratp_info[i] = get_individual_info(dvf_coords[i], ratp_coords)
    except:
        ratp_info[i] = (np.nan, np.nan, np.nan, np.nan)

100%|███████████████████████████████████████████████████████████████████████| 163363/163363 [12:42:48<00:00,  3.57it/s]

Wall time: 12h 42min 48s


In [11]:
dvf_dist_ratp = (dvf_geocoordinates
 .loc[:, ['adresse', 'postcode', 'latitude', 'longitude']]
 .assign(ratp_ngare1 = [x[0] for x in ratp_info],
         ratp_ngare2 = [x[1] for x in ratp_info],
         ratp_ngare3 = [x[2] for x in ratp_info],
         ratp_min_distance = [x[3] for x in ratp_info])
)

In [28]:
dvf_geocode = pd.read_csv('dvf-data.csv')

C:\Users\JouJo\Anaconda3\envs\Python3_9\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (17,24) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [29]:
dvf_geocode = (dvf_geocode
 .assign(
     nvoie = lambda x: x.no_voie.astype('int').astype('str'),
     adresse = lambda x: x.nvoie + ' ' + x.type_de_voie + ' ' + x.voie,
     )
 .loc[:, ['adresse']]
)

In [23]:
dvf_dist_ratp = dvf_dist_ratp.drop_duplicates(subset=['adresse'])

In [33]:
dvf_geocode_clean = pd.merge(dvf_geocode, dvf_dist_ratp, on='adresse')

In [36]:
dvf_dist_clean = dvf_dist_ratp.drop_duplicates(subset='adresse')

In [37]:
dvf_dist_clean.to_csv('dvf-dist-ratp.csv')